# QSVM for Breast Cancer Detection that will not only be created for utility, but for comparison to a SVM of each ones efficiency.

## Niko Tsiolas, July 2024

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZZFeatureMap
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.sampler import SamplerV2


# Load the data 
data = pd.read_csv('Breast_Cancer_Data.csv')

# Separating the features and target 
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Set up the quantum feature map
feature_dim = X_train_scaled.shape[1]  
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2)

# Set up the IBM Quantum service
service = QiskitRuntimeService(channel="ibm_quantum", token="a13aef24d4de13cbf76251ab4de79999ef204324397a0dcefd3c1cc8bdb0ae66643a03e3de6f00139f1e247ed78be6ef79e384119795d4b83fabae4616e29705")

In [1]:
# Use FakeManilaV2 as our backend
fake_manila = FakeManilaV2()

# Generate a preset pass manager for the fake backend
pm = generate_preset_pass_manager(backend=fake_manila, optimization_level=1)

# Set up options for the simulator
options = {"simulator": {"seed_simulator": 42}}

# Create a Sampler using the fake backend
sampler = SamplerV2(backend=fake_manila, options=options)

# Set up the quantum kernel
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

# Create and train the QSVM
qsvm = QSVC(quantum_kernel=quantum_kernel)

print("Training QSVM...")
qsvm.fit(X_train_scaled, y_train)

print("Making predictions...")
y_pred = qsvm.predict(X_test_scaled)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Script execution completed.")

NameError: name 'FakeManilaV2' is not defined

In [34]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("\nClassication Report:")
print(classification_report(y_test, y_pred))

NameError: name 'y_pred' is not defined